In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

***import all library***

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split
from scipy import stats

***reading the train data***

In [ ]:
train=pd.read_csv("../input/house-prices-advanced-regression-techniques/train.csv")
train.head(5)

***reading the test data***

In [ ]:
test=pd.read_csv("../input/house-prices-advanced-regression-techniques/test.csv")
test.head(5)

In [ ]:
train.describe()

In [ ]:
missing_value=train.isna().sum().sort_values(ascending=False)
missing_value.head(20)

***drop all columns where missing value greater than 5***

In [ ]:
train.drop(missing_value[missing_value>5].index,axis=1,inplace=True)

In [ ]:
train.shape

***set feature variable***

In [ ]:
X=train

***set target variable***

In [ ]:
Y=X.SalePrice

In [ ]:
normalize=stats.boxcox(Y)
fig,ax=plt.subplots(1,2)
sns.distplot(Y,ax=ax[0])
ax[0].set_title("Original")
sns.distplot(normalize[0],ax=ax[1])
ax[1].set_title("Normalize")

***correlation between features variable and Target variables***

In [ ]:
Correlation=X.corr()
high_corr=Correlation.index[Correlation['SalePrice']>.5]
plt.figure(figsize=(10,10))
sns.heatmap(X[high_corr].corr(),annot=True)

In [ ]:
high_corr

***set the features variable which is greater correlation respect to target variable(SalePrice)***

In [ ]:
features=['OverallQual', 'YearBuilt', 'YearRemodAdd', 'TotalBsmtSF', '1stFlrSF',
       'GrLivArea', 'FullBath', 'TotRmsAbvGrd', 'GarageCars', 'GarageArea']

In [ ]:
for i in features:
    sns.lmplot(x='SalePrice',y=i,data=X)

In [ ]:
test_id = test['Id']
total_data = pd.concat([X, test], axis=0, sort=False)
total_data = total_data.drop(['Id', 'SalePrice'], axis=1)

In [ ]:
missing_values_count = total_data.isna().sum().sort_values(ascending=False)
missing_values_count.head(20)

In [ ]:
total_data.drop(missing_data[missing_data['missing']>5].index,axis=1,inplace=True)

In [ ]:
total_data.info()

***Label Encoder***

In [ ]:
from sklearn.preprocessing import LabelEncoder
object_cols = [col for col in total_data.columns if total_data[col].dtype == "object"]
label_encoder=LabelEncoder()
for col in object_cols:
    total_data[col]=label_encoder.fit_transform(total_data[col])
total_data.info()

***imputer method for numeric value***

In [ ]:
from sklearn.impute import SimpleImputer
numeric_cols = [col for col in total_data.columns if total_data[col].dtype in ['int64','float64']]
my_imputer=SimpleImputer(strategy='median')
data = pd.DataFrame(my_imputer.fit_transform(total_data))
data.columns = total_data.columns
a=data.isnull().sum().sort_values(ascending=False)
a.head(25)

In [ ]:
x_train =data[:len(Y)]
x_test = data[len(Y):]
x_train.info()
x_test.info()

In [ ]:
X_train, X_valid, y_train, y_valid = train_test_split(x_train, Y, train_size=0.8, test_size=0.2, random_state=1)

In [ ]:
model = RandomForestRegressor(n_estimators=200, random_state=1)
model.fit(X_train, y_train)
preds = model.predict(X_valid)
print(mean_absolute_error(y_valid, preds))

In [ ]:
preds_test =model.predict(x_test)
print(preds_test)

In [ ]:
sub = pd.DataFrame()
sub['Id'] = test_id
sub['SalePrice'] = preds_test
sub.to_csv('mysubmission.csv',index=False)
print(sub)